In [ ]:
import pandas as pd
from modules.create_holidays_functions import *

In [3]:
df_holidays = pd.DataFrame()

In [4]:
today = datetime.now()
beginning_date = today - timedelta(days = 43)
end_date = today + timedelta(days = 655)

In [5]:
current_date = beginning_date
while current_date <= end_date:
    row = pd.DataFrame([[current_date.date(), '']], columns=['date', 'vacances_nantes']) 
    df_holidays = pd.concat([df_holidays, row], ignore_index=True)
    current_date += timedelta(days=1) 

In [6]:
df_holidays.head()

,date,vacances_nantes
0,2025-02-01,
1,2025-02-02,
2,2025-02-03,
3,2025-02-04,
4,2025-02-05,


In [7]:
df_holidays['vacances_nantes'] = df_holidays['date'].apply(define_holiday)

In [8]:
df_holidays['jour_ferie'] = df_holidays['date'].apply(define_public_holiday)

In [10]:
df_holidays.head(25)

,date,vacances_nantes,jour_ferie
0,2025-02-01,0,0
1,2025-02-02,0,0
2,2025-02-03,0,0
3,2025-02-04,0,0
4,2025-02-05,0,0
5,2025-02-06,0,0
6,2025-02-07,0,0
7,2025-02-08,1,0
8,2025-02-09,1,0
9,2025-02-10,1,0


In [11]:
df_holidays.to_csv('/Users/manu/Desktop/SUP/Projet 2/AI_model_urban_mobility/data/preprocessed/holidays_public_holidays.csv', index=False)